# 05. Lagrange Multiplier 완전 정복

## 학습 목표
- Constrained optimization의 필요성
- Lagrange multiplier 방법의 기하학적 의미
- Equality/Inequality constraints 처리
- KKT 조건 완전 이해
- 다양한 예제 문제 풀이

**참고 자료**: ML_L14a/b_Constrained.Optimization

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize
import seaborn as sns

sns.set_style('whitegrid')
np.random.seed(42)

## 1. Unconstrained vs Constrained Optimization

### 1.1 Unconstrained Optimization

$$\min_{x} f(x)$$

**해법**: $\nabla f(x^*) = 0$ (critical point)

### 1.2 Constrained Optimization

$$\begin{align}
\min_{x} \quad & f(x) \\
\text{subject to} \quad & g(x) = 0
\end{align}$$

**문제**: $\nabla f(x) = 0$이 아닐 수 있음!

**예**: 원 위에서 함수 최소화

In [ ]:
# 시각화: Constrained vs Unconstrained

# 목적 함수: f(x, y) = x^2 + y^2
x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Unconstrained
ax = axes[0]
contour = ax.contour(X, Y, Z, levels=20, cmap='viridis')
ax.plot(0, 0, 'r*', markersize=20, label='Minimum (0, 0)')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Unconstrained: min(x²+y²)')
ax.legend()
ax.grid(alpha=0.3)
ax.axis('equal')

# Constrained: x + y = 1
ax = axes[1]
contour = ax.contour(X, Y, Z, levels=20, cmap='viridis')
# Constraint line
x_line = np.linspace(-1, 2, 100)
y_line = 1 - x_line
ax.plot(x_line, y_line, 'r-', linewidth=3, label='Constraint: x+y=1')
# Constrained minimum: (0.5, 0.5)
ax.plot(0.5, 0.5, 'r*', markersize=20, label='Constrained min')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Constrained: min(x²+y²) s.t. x+y=1')
ax.legend()
ax.grid(alpha=0.3)
ax.axis('equal')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

plt.tight_layout()
plt.show()

print("Unconstrained minimum: (0, 0), f = 0")
print("Constrained minimum: (0.5, 0.5), f = 0.5")
print("\n→ Constraint가 있으면 minimum이 달라진다!")

## 2. Lagrange Multiplier 방법 (Equality Constraints)

### 2.1 문제 설정

$$\begin{align}
\min_{x} \quad & f(x) \\
\text{subject to} \quad & g(x) = 0
\end{align}$$

### 2.2 Lagrangian 함수

$$\mathcal{L}(x, \lambda) = f(x) + \lambda g(x)$$

where $\lambda$ is the **Lagrange multiplier**

### 2.3 최적성 조건 (Optimality Conditions)

Minimum에서:

1. **Stationarity**: $\nabla_x \mathcal{L} = \nabla f(x^*) + \lambda^* \nabla g(x^*) = 0$
2. **Feasibility**: $g(x^*) = 0$

### 2.4 기하학적 해석

**핵심**: Minimum에서 $\nabla f$와 $\nabla g$가 **평행**!

$$\nabla f(x^*) = -\lambda^* \nabla g(x^*)$$

**이유**: 
- $\nabla g$: Constraint surface의 법선 (normal vector)
- $\nabla f$: 함수가 가장 빠르게 증가하는 방향
- Minimum에서는 constraint를 따라 이동해도 f가 증가 → $\nabla f \perp$ tangent → $\nabla f \parallel \nabla g$

In [ ]:
# 기하학적 해석 시각화

# f(x, y) = x^2 + y^2, g(x, y) = x + y - 1 = 0

x_opt, y_opt = 0.5, 0.5  # Constrained minimum

# Gradients
grad_f = np.array([2*x_opt, 2*y_opt])  # ∇f = (2x, 2y)
grad_g = np.array([1, 1])  # ∇g = (1, 1)

plt.figure(figsize=(10, 8))

# Contours
x = np.linspace(-1, 2, 100)
y = np.linspace(-1, 2, 100)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2
plt.contour(X, Y, Z, levels=15, cmap='coolwarm', alpha=0.6)

# Constraint
x_line = np.linspace(-0.5, 2, 100)
y_line = 1 - x_line
plt.plot(x_line, y_line, 'b-', linewidth=3, label='Constraint: x+y=1')

# Minimum point
plt.plot(x_opt, y_opt, 'r*', markersize=20, label=f'Minimum ({x_opt}, {y_opt})')

# Gradients
scale = 0.3
plt.arrow(x_opt, y_opt, scale*grad_f[0], scale*grad_f[1], 
         head_width=0.1, head_length=0.1, fc='red', ec='red', linewidth=2)
plt.text(x_opt + scale*grad_f[0] + 0.1, y_opt + scale*grad_f[1], 
        r'$\nabla f$', fontsize=16, color='red', fontweight='bold')

plt.arrow(x_opt, y_opt, scale*grad_g[0], scale*grad_g[1], 
         head_width=0.1, head_length=0.1, fc='blue', ec='blue', linewidth=2)
plt.text(x_opt + scale*grad_g[0] + 0.1, y_opt + scale*grad_g[1] - 0.2, 
        r'$\nabla g$', fontsize=16, color='blue', fontweight='bold')

plt.xlabel('x', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.title(r'Geometric Interpretation: $\nabla f \parallel \nabla g$ at minimum', 
         fontsize=16, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.axis('equal')
plt.xlim(-0.5, 2)
plt.ylim(-0.5, 2)
plt.show()

print("=== 최적점에서 ===")
print(f"∇f = {grad_f}")
print(f"∇g = {grad_g}")
print(f"\n∇f / ∇g = {grad_f / grad_g}")
print("→ 비율이 같음 = 평행!")
print(f"\nλ = -{grad_f[0] / grad_g[0]:.2f}")

## 3. 예제 1: 간단한 2D 문제

### 문제

$$\begin{align}
\min_{x, y} \quad & f(x, y) = x^2 + y^2 \\
\text{subject to} \quad & g(x, y) = x + y - 1 = 0
\end{align}$$

### 풀이

**Step 1**: Lagrangian

$$\mathcal{L}(x, y, \lambda) = x^2 + y^2 + \lambda(x + y - 1)$$

**Step 2**: 편미분 = 0

$$\frac{\partial \mathcal{L}}{\partial x} = 2x + \lambda = 0 \quad \Rightarrow \quad x = -\frac{\lambda}{2}$$

$$\frac{\partial \mathcal{L}}{\partial y} = 2y + \lambda = 0 \quad \Rightarrow \quad y = -\frac{\lambda}{2}$$

$$\frac{\partial \mathcal{L}}{\partial \lambda} = x + y - 1 = 0$$

**Step 3**: 연립방정식 풀기

$x = y$이고 $x + y = 1$이므로:

$$2x = 1 \quad \Rightarrow \quad x = y = \frac{1}{2}$$

$$\lambda = -2x = -1$$

**결과**: $(x^*, y^*) = (0.5, 0.5)$, $\lambda^* = -1$, $f^* = 0.5$

In [ ]:
# 예제 1 검증

# Scipy로 풀기
from scipy.optimize import minimize

# 목적 함수
def objective(vars):
    x, y = vars
    return x**2 + y**2

# Constraint
def constraint(vars):
    x, y = vars
    return x + y - 1

# 제약 조건 정의
cons = {'type': 'eq', 'fun': constraint}

# 초기값
x0 = [0, 0]

# 최적화
result = minimize(objective, x0, method='SLSQP', constraints=cons)

print("=== Scipy 결과 ===")
print(f"Optimal x, y: {result.x}")
print(f"Optimal f: {result.fun:.4f}")
print(f"\n해석적 풀이: (0.5, 0.5), f = 0.5")
print(f"일치 여부: {np.allclose(result.x, [0.5, 0.5])}")

## 4. 예제 2: 다변수 최적화

### 문제

$$\begin{align}
\min_{x, y, z} \quad & f(x, y, z) = x^2 + 2y^2 + 3z^2 \\
\text{subject to} \quad & g(x, y, z) = x + y + z - 1 = 0
\end{align}$$

### 풀이

**Lagrangian**:

$$\mathcal{L} = x^2 + 2y^2 + 3z^2 + \lambda(x + y + z - 1)$$

**KKT conditions**:

$$\frac{\partial \mathcal{L}}{\partial x} = 2x + \lambda = 0 \quad \Rightarrow \quad x = -\frac{\lambda}{2}$$

$$\frac{\partial \mathcal{L}}{\partial y} = 4y + \lambda = 0 \quad \Rightarrow \quad y = -\frac{\lambda}{4}$$

$$\frac{\partial \mathcal{L}}{\partial z} = 6z + \lambda = 0 \quad \Rightarrow \quad z = -\frac{\lambda}{6}$$

$$x + y + z = 1$$

**대입**:

$$-\frac{\lambda}{2} - \frac{\lambda}{4} - \frac{\lambda}{6} = 1$$

$$-\lambda\left(\frac{6 + 3 + 2}{12}\right) = 1$$

$$-\lambda \cdot \frac{11}{12} = 1 \quad \Rightarrow \quad \lambda = -\frac{12}{11}$$

**해**:

$$x^* = \frac{6}{11}, \quad y^* = \frac{3}{11}, \quad z^* = \frac{2}{11}$$

In [ ]:
# 예제 2 검증

def objective2(vars):
    x, y, z = vars
    return x**2 + 2*y**2 + 3*z**2

def constraint2(vars):
    x, y, z = vars
    return x + y + z - 1

cons2 = {'type': 'eq', 'fun': constraint2}
x0_2 = [0, 0, 0]

result2 = minimize(objective2, x0_2, method='SLSQP', constraints=cons2)

print("=== 예제 2 검증 ===")
print(f"Scipy 결과: {result2.x}")
print(f"해석 solution: [{6/11:.4f}, {3/11:.4f}, {2/11:.4f}]")
print(f"\nOptimal f (Scipy): {result2.fun:.4f}")

# 직접 계산
x_star = 6/11
y_star = 3/11
z_star = 2/11
f_star = x_star**2 + 2*y_star**2 + 3*z_star**2
print(f"Optimal f (해석): {f_star:.4f}")

# 제약 확인
print(f"\nConstraint check: x+y+z = {x_star + y_star + z_star:.4f}")

## 5. Inequality Constraints

### 5.1 문제

$$\begin{align}
\min_{x} \quad & f(x) \\
\text{subject to} \quad & g_i(x) \leq 0, \quad i = 1, \ldots, m
\end{align}$$

### 5.2 KKT 조건 (Karush-Kuhn-Tucker)

**Lagrangian**:

$$\mathcal{L}(x, \mu) = f(x) + \sum_{i=1}^{m} \mu_i g_i(x)$$

**KKT 조건** (필요조건):

1. **Stationarity**: $\nabla_x \mathcal{L} = 0$
2. **Primal feasibility**: $g_i(x) \leq 0$, $\forall i$
3. **Dual feasibility**: $\mu_i \geq 0$, $\forall i$
4. **Complementary slackness**: $\mu_i g_i(x) = 0$, $\forall i$

### 5.3 Complementary Slackness 해석

각 constraint $i$에 대해 **둘 중 하나**:
- $\mu_i = 0$: Constraint가 **inactive** (여유 있음)
- $g_i(x) = 0$: Constraint가 **active** (경계)

**Active constraint**: Equality처럼 작동!  
**Inactive constraint**: 무시 가능!

## 6. 예제 3: Inequality Constraint

### 문제

$$\begin{align}
\min_{x, y} \quad & f(x, y) = (x - 2)^2 + (y - 2)^2 \\
\text{subject to} \quad & g_1(x, y) = x + y - 1 \leq 0 \\
& g_2(x, y) = -x \leq 0 \quad (x \geq 0) \\
& g_3(x, y) = -y \leq 0 \quad (y \geq 0)
\end{align}$$

**직관**: 점 (2, 2)에서 가장 가까운 삼각형 영역의 점

In [ ]:
# 예제 3 시각화

fig, ax = plt.subplots(figsize=(10, 8))

# Contours of f(x, y) = (x-2)^2 + (y-2)^2
x = np.linspace(-0.5, 3, 200)
y = np.linspace(-0.5, 3, 200)
X, Y = np.meshgrid(x, y)
Z = (X - 2)**2 + (Y - 2)**2
contour = ax.contour(X, Y, Z, levels=20, cmap='viridis', alpha=0.6)
ax.clabel(contour, inline=True, fontsize=10)

# Feasible region
# x >= 0, y >= 0, x + y <= 1
vertices = np.array([[0, 0], [1, 0], [0, 1], [0, 0]])
ax.fill(vertices[:, 0], vertices[:, 1], alpha=0.3, color='green', label='Feasible region')
ax.plot(vertices[:, 0], vertices[:, 1], 'g-', linewidth=2)

# Unconstrained minimum
ax.plot(2, 2, 'b*', markersize=20, label='Unconstrained min (2, 2)')

# Constrained minimum (예상: (0.5, 0.5))
ax.plot(0.5, 0.5, 'r*', markersize=20, label='Constrained min (0.5, 0.5)')

ax.set_xlabel('x', fontsize=14)
ax.set_ylabel('y', fontsize=14)
ax.set_title('Inequality Constraints Example', fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(alpha=0.3)
ax.set_xlim(-0.5, 3)
ax.set_ylim(-0.5, 3)
plt.show()

In [ ]:
# 예제 3 풀이

def objective3(vars):
    x, y = vars
    return (x - 2)**2 + (y - 2)**2

# Inequality constraints: g(x) <= 0
constraints3 = [
    {'type': 'ineq', 'fun': lambda vars: 1 - vars[0] - vars[1]},  # x + y <= 1
    {'type': 'ineq', 'fun': lambda vars: vars[0]},  # x >= 0
    {'type': 'ineq', 'fun': lambda vars: vars[1]}   # y >= 0
]

x0_3 = [0.1, 0.1]
result3 = minimize(objective3, x0_3, method='SLSQP', constraints=constraints3)

print("=== 예제 3: Inequality Constraints ===")
print(f"Optimal (x, y): {result3.x}")
print(f"Optimal f: {result3.fun:.4f}")
print(f"\n제약 확인:")
print(f"  x + y = {result3.x[0] + result3.x[1]:.4f} (should be ≤ 1)")
print(f"  x = {result3.x[0]:.4f} (should be ≥ 0)")
print(f"  y = {result3.x[1]:.4f} (should be ≥ 0)")

# Active constraints 확인
print("\nActive constraints:")
g1 = 1 - result3.x[0] - result3.x[1]
print(f"  g1 (x+y-1): {g1:.6f}")
if abs(g1) < 1e-4:
    print("    → ACTIVE! (x + y = 1)")
else:
    print("    → Inactive")

## 7. 연습문제 1: Portfolio Optimization

### 문제

2개 자산에 투자. $x_1$, $x_2$는 각 자산의 투자 비율.

**목표**: Risk 최소화 (variance)

$$\begin{align}
\min_{x_1, x_2} \quad & f(x_1, x_2) = x_1^2 + x_2^2 + x_1 x_2 \\
\text{subject to} \quad & x_1 + x_2 = 1 \quad (\text{전액 투자}) \\
& x_1, x_2 \geq 0 \quad (\text{공매도 금지})
\end{align}$$

### 풀이

**Lagrangian** (equality만):

$$\mathcal{L} = x_1^2 + x_2^2 + x_1 x_2 + \lambda(x_1 + x_2 - 1)$$

**KKT**:

$$\frac{\partial \mathcal{L}}{\partial x_1} = 2x_1 + x_2 + \lambda = 0$$

$$\frac{\partial \mathcal{L}}{\partial x_2} = 2x_2 + x_1 + \lambda = 0$$

$$x_1 + x_2 = 1$$

첫 두 식을 빼면:

$$2x_1 + x_2 - 2x_2 - x_1 = 0 \quad \Rightarrow \quad x_1 = x_2$$

대입:

$$2x_1 = 1 \quad \Rightarrow \quad x_1 = x_2 = 0.5$$

In [ ]:
# 연습문제 1 풀이

def portfolio_obj(x):
    x1, x2 = x
    return x1**2 + x2**2 + x1*x2

cons_portfolio = [
    {'type': 'eq', 'fun': lambda x: x[0] + x[1] - 1},  # x1 + x2 = 1
    {'type': 'ineq', 'fun': lambda x: x[0]},  # x1 >= 0
    {'type': 'ineq', 'fun': lambda x: x[1]}   # x2 >= 0
]

x0_port = [0.5, 0.5]
result_port = minimize(portfolio_obj, x0_port, method='SLSQP', constraints=cons_portfolio)

print("=== Portfolio Optimization ===")
print(f"Optimal allocation: x1 = {result_port.x[0]:.4f}, x2 = {result_port.x[1]:.4f}")
print(f"Minimum risk: {result_port.fun:.4f}")
print(f"\n해석: 50-50으로 분산 투자가 최적!")

## 8. 연습문제 2: Box Optimization

### 문제

직육면체 상자를 만든다. 길이 $x$, 너비 $y$, 높이 $z$.

**목표**: 부피 최대화

**제약**: 표면적이 12 이하

$$\begin{align}
\max_{x,y,z} \quad & V = xyz \\
\text{subject to} \quad & 2(xy + xz + yz) \leq 12 \\
& x, y, z \geq 0
\end{align}$$

**변환**: $\max V = -\min (-V)$

In [ ]:
# 연습문제 2 풀이

def box_obj(vars):
    x, y, z = vars
    return -(x * y * z)  # Negative for minimization

cons_box = [
    {'type': 'ineq', 'fun': lambda v: 12 - 2*(v[0]*v[1] + v[ 0]*v[2] + v[1]*v[2])},  # Surface area
    {'type': 'ineq', 'fun': lambda v: v[0]},  # x >= 0
    {'type': 'ineq', 'fun': lambda v: v[1]},  # y >= 0
    {'type': 'ineq', 'fun': lambda v: v[2]}   # z >= 0
]

x0_box = [1, 1, 1]
result_box = minimize(box_obj, x0_box, method='SLSQP', constraints=cons_box)

print("=== Box Optimization ===")
print(f"Optimal dimensions: x = {result_box.x[0]:.4f}, y = {result_box.x[1]:.4f}, z = {result_box.x[2]:.4f}")
print(f"Maximum volume: {-result_box.fun:.4f}")

# 표면적 확인
x, y, z = result_box.x
surface_area = 2*(x*y + x*z + y*z)
print(f"Surface area: {surface_area:.4f} (should be ≤ 12)")

# 직관: 정육면체일 때 최대!
print(f"\n해석: x ≈ y ≈ z → 정육면체가 최적!")

## 9. 족보 문제

### 문제 (기출 변형)

$$\begin{align}
\min_{x,y} \quad & f(x, y) = x^2 + 4y^2 \\
\text{subject to} \quad & 2x + y = 4
\end{align}$$

### 상세 풀이

**Step 1**: Lagrangian

$$\mathcal{L}(x, y, \lambda) = x^2 + 4y^2 + \lambda(2x + y - 4)$$

**Step 2**: KKT Stationarity

$$\begin{align}
\frac{\partial \mathcal{L}}{\partial x} &= 2x + 2\lambda = 0 \quad &\Rightarrow x = -\lambda \\
\frac{\partial \mathcal{L}}{\partial y} &= 8y + \lambda = 0 \quad &\Rightarrow y = -\frac{\lambda}{8} \\
\frac{\partial \mathcal{L}}{\partial \lambda} &= 2x + y - 4 = 0
\end{align}$$

**Step 3**: 대입

$$2(-\lambda) + \left(-\frac{\lambda}{8}\right) = 4$$

$$-2\lambda - \frac{\lambda}{8} = 4$$

$$-\lambda\left(2 + \frac{1}{8}\right) = 4$$

$$-\lambda \cdot \frac{17}{8} = 4$$

$$\lambda = -\frac{32}{17}$$

**Step 4**: 해

$$x^* = -\lambda = \frac{32}{17}, \quad y^* = -\frac{\lambda}{8} = \frac{4}{17}$$

$$f^* = \left(\frac{32}{17}\right)^2 + 4\left(\frac{4}{17}\right)^2 = \frac{1024 + 64}{289} = \frac{1088}{289} \approx 3.76$$

In [ ]:
# 족보 문제 검증

def exam_obj(vars):
    x, y = vars
    return x**2 + 4*y**2

def exam_cons(vars):
    x, y = vars
    return 2*x + y - 4

cons_exam = {'type': 'eq', 'fun': exam_cons}
x0_exam = [1, 1]

result_exam = minimize(exam_obj, x0_exam, method='SLSQP', constraints=cons_exam)

print("=== 족보 문제 ===")
print(f"Scipy 결과: x = {result_exam.x[0]:.4f}, y = {result_exam.x[1]:.4f}")
print(f"해석 결과: x = {32/17:.4f}, y = {4/17:.4f}")
print(f"\nOptimal f (Scipy): {result_exam.fun:.4f}")
print(f"Optimal f (해석): {1088/289:.4f}")
print(f"\n제약 확인: 2x + y = {2*result_exam.x[0] + result_exam.x[1]:.4f}")

## 10. 요약 정리

### Lagrange Multiplier 방법

| 단계 | 내용 |
|------|------|
| 1 | Lagrangian 구성: $\mathcal{L} = f(x) + \sum \lambda_i g_i(x)$ |
| 2 | 편미분 = 0: $\nabla_x \mathcal{L} = 0$, $\nabla_{\lambda} \mathcal{L} = 0$ |
| 3 | 연립방정식 풀기 |
| 4 | 해 검증 (constraint 확인) |

### KKT 조건 (Inequality)

1. **Stationarity**: $\nabla f(x^*) + \sum \mu_i \nabla g_i(x^*) = 0$
2. **Primal feasibility**: $g_i(x^*) \leq 0$
3. **Dual feasibility**: $\mu_i \geq 0$
4. **Complementary slackness**: $\mu_i g_i(x^*) = 0$

### 핵심 직관

- **Equality**: $\nabla f \parallel \nabla g$ (평행)
- **Inequality**: Active constraint는 equality처럼, Inactive는 무시
- **λ (multiplier)**: Constraint의 "가치" (shadow price)

### 다음 단계

- SVM에서 Lagrange 적용 복습
- Convex optimization 심화
- Interior point methods